<a href="https://colab.research.google.com/github/xxotto/covid19-peru/blob/main/Notebooks/3_DataFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obteniendo más información de cada caso positivo por COVID-19 en Perú

En este notebook se busca obtener un dataset de todos los **casos positivos** reportados por COVID-19, y agregar información sobre fallecimiento y vacunación recibida (mediante **"UUID"**). Es decir se busca unir las 3 bases de datos, de casos positivos, fallecimientos y vacunación recibida en una sola.

Se considera como **vacunado** a todas las personas que recibieron 2 dosis de vacunación.

## 1. Preparación

### 1.1 Descargar fuentes de datos

Las fuentes de datos pueden ser descargadas a través de la [Plataforma Nacional de Datos Abiertos](https://www.datosabiertos.gob.pe/) del Gobierno del Perú.

1. [Fallecidos por COVID-19](https://www.datosabiertos.gob.pe/dataset/fallecidos-por-covid-19-ministerio-de-salud-minsa)
2. [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion)
3. [Casos positivos por COVID-19](https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa)
4. [Centros de vacunación](https://www.datosabiertos.gob.pe/dataset/centros-de-vacunacion)
5. [Códigos equivalentes de UBIGEO del Perú](https://www.datosabiertos.gob.pe/dataset/codigos-equivalentes-de-ubigeo-del-peru)

**Nota 1:** En este caso todos los csv se encuentran en google drive, usted puede descargarlos desde los enlaces correspondientes y colocarlos en la dirección de RawData. Una vez realizado esto saltar al paso 2.

**Nota 2:** Puede que hayan habido cambios en las bases de datos usadas ya que son actualizadas a diario (actualmente), por tanto cambios en el código pueden ser necesarios más adelante.


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

%cd /content/drive/MyDrive/GitHub/covid19-peru/Notebooks/
%ls

Mounted at /content/drive/
/content/drive/MyDrive/GitHub/covid19-peru/Notebooks
0_ExploratoryDataAnalysis.ipynb  3_DataFiltering.ipynb  6_DataFiltering.ipynb
1_DataFiltering.ipynb            4_DataFiltering.ipynb  DataProblems.ipynb
2_DataFiltering.ipynb            5_DataFiltering.ipynb  functions.py


### 1.2 Instalar librerías necesarias

In [ ]:
!pip install epiweeks
!pip install dask

In [ ]:
!python -m pip install dask[dataframe] --upgrade

     |████████████████████████████████| 136 kB 4.8 MB/s 


## 2. Lectura y Procesamiento

### Cargar Librerías

In [ ]:
import pandas as pd
import numpy as np
from epiweeks import Week
import dask.dataframe as dd
from datetime import datetime

### Diccionarios para codificar variables

In [ ]:
dic_sex = {'MASCULINO': 1,'FEMENINO':2}
dic_met = {'PR': 1, 'PCR':2, 'AG':3}
dic_dep = {
    "AMAZONAS":1,
    "ANCASH":2,
    "APURIMAC":3,   
    "AREQUIPA":4,
    "AYACUCHO":5,   
    "CAJAMARCA":6,
    "CALLAO":7,
    "CUSCO":8,
    "HUANCAVELICA":9,	
    "HUANUCO":10,	
    "ICA":11,	
    "JUNIN":12,
    "LA LIBERTAD":13,
    "LAMBAYEQUE":14,
    "LIMA":15,
    "LORETO":16,
    "MADRE DE DIOS":17,
    "MOQUEGUA":18,	
    "PASCO":19,
    "PIURA":20,
    "PUNO":21,
    "SAN MARTIN":22,
    "TACNA":23,
    "TUMBES":24,
    "UCAYALI":25
}

### Dataset de casos positivos

In [ ]:
# Leer dataset
url_cas = '../RawData/positivos_covid.csv'
col_cas = ['FECHA_RESULTADO','EDAD', 'DEPARTAMENTO', 'SEXO', 'id_persona', 'METODODX']
df_cas = pd.read_csv(url_cas, sep = ';', usecols = col_cas, dtype={'EDAD': 'float64', 
                                                                   'FECHA_RESULTADO': 'float64'})

In [ ]:
df_cas.head()

,DEPARTAMENTO,METODODX,EDAD,SEXO,FECHA_RESULTADO,id_persona
0,CALLAO,AG,27.0,MASCULINO,20220131.0,24662494.0
1,UCAYALI,AG,39.0,FEMENINO,20220127.0,24662689.0
2,LIMA,PR,25.0,MASCULINO,20201217.0,24662153.0
3,AREQUIPA,PCR,27.0,MASCULINO,20220120.0,24662155.0
4,ICA,PR,20.0,FEMENINO,20200822.0,24662175.0


In [ ]:
# TOTAL de casos positivos
len(df_cas)

3534686

In [ ]:
# Remover casos con información faltante
df_cas = df_cas.dropna()

# TOTAL de casos positivos sin 'missing values'
len(df_cas)

3465000

In [ ]:
# Cambiar 'data type' para reducir tamaño
df_cas = df_cas.astype({'EDAD':'int8', 'FECHA_RESULTADO': 'int32', 'id_persona': 'int32'})

# Correguir datos erróneos encontrados en el anáisis exploradorio de "0_ExploratoryDataAnalysis.ipynb"
df_cas.drop(df_cas[df_cas.SEXO == '.'].index, inplace = True)
df_cas.drop(df_cas[df_cas.EDAD == -79].index, inplace = True)  # un -9 en edad

In [ ]:
# TOTAL de casos positivos sin valores erróneos
len(df_cas)

3464999

In [ ]:
# Categorizar edad en grupos etáreos
def edad_cat(df, col):
    """Función para categorizar edad en grupos etáreos"""
    conditions = [(df[col]< 18),
                  (df[col]>=18) & (df[col]<30),
                  (df[col]>=30) & (df[col]<40),
                  (df[col]>=40) & (df[col]<50),
                  (df[col]>=50) & (df[col]<60),
                  (df[col]>=60) & (df[col]<70),
                  (df[col]>=70) & (df[col]<80),
                  (df[col]>=80)]
    choices = [0,1,2,3,4,5,6,7]
    return np.select(conditions, choices, default=np.nan).astype('int8')
df_cas['edad_cat'] = edad_cat(df_cas, 'EDAD') 

# Reemplazar sexo con un code
df_cas['SEXO'].replace(dic_sex, inplace=True)
df_cas['SEXO'] = df_cas['SEXO'].astype('int8')

# Ordenar columnas
df_cas.rename(columns = {'FECHA_RESULTADO':'fecha_resultado',
                         'EDAD': 'edad',
                         'SEXO': 'sexo',
                         'DEPARTAMENTO': 'departamento',
                         'METODODX': 'metododx'}, inplace=True)
df_cas = df_cas[['id_persona','fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento']] \
         .sort_values(by = ['fecha_resultado']) \
         .reset_index(drop = True)

# Codificar variables
df_cas['departamento'] = df_cas['departamento'].map(dic_dep).astype('float')  
df_cas['metododx'].replace(dic_met, inplace=True)
df_cas['metododx'] = df_cas['metododx'].astype('int8')

In [ ]:
df_cas.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento
0,34978840,20200306,2,1,25,1,15.0
1,12149681,20200307,2,1,25,1,15.0
2,12211040,20200307,2,1,7,0,15.0
3,29640291,20200307,2,1,29,1,4.0
4,7800643,20200307,2,2,74,6,15.0


### Dataset de fallecidos

In [ ]:
# Leer el dataset
url_fal = '../RawData/fallecidos_covid.csv'
col_fal = ['FECHA_FALLECIMIENTO', 'UUID']
df_fal = pd.read_csv(url_fal, sep = ';', usecols= col_fal, dtype = {'FECHA_FALLECIMIENTO':'int32'}, 
                     low_memory=False)

In [ ]:
df_fal.head()

,FECHA_FALLECIMIENTO,UUID
0,20210611,24833991.0
1,20210317,24761117.0
2,20210602,24767070.0
3,20210703,24751741.0
4,20210506,24671820.0


In [ ]:
# TOTAL de casos positivos
len(df_fal)

211423

In [ ]:
# Remover casos con información faltante
df_fal = df_fal.dropna()

# TOTAL de casos positivos sin 'missing values'
len(df_fal)

207538

In [ ]:
# Nombrar a todos los casos de fallecimientos con un identificador
df_fal['fallecido'] = 1

# Cambiar de data type para reducir tamaño
df_fal = df_fal.astype({'UUID': 'int32', 'fallecido': 'int8'})

# Transformar a formato fecha para operar los días
df_fal['fecha'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')

# Agregar la semana epidemiológica y el año
df_fal[['year','epi_week']] = df_fal['fecha'].apply(lambda date : Week.fromdate(date) \
                                                                      .weektuple()) \
                                                                      .tolist()

# Ordenar columnas
df_fal.rename(columns = {'FECHA_FALLECIMIENTO':'fecha_fallecimiento', 'UUID':'id_persona'}, inplace=True)

df_fal.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,fecha_fallecimiento,id_persona,fallecido,fecha,year,epi_week
0,20210611,24833991,1,2021-06-11,2021,23
1,20210317,24761117,1,2021-03-17,2021,11
2,20210602,24767070,1,2021-06-02,2021,22
3,20210703,24751741,1,2021-07-03,2021,26
4,20210506,24671820,1,2021-05-06,2021,18


### Buscar en el dataset de casos positivos quienes fallecieron y cuándo

In [ ]:
# Unir el dataframde de casos positivos con el dataset de fallecidos mediante 'id_persona'
# {‘left’, ‘right’, ‘outer’, ‘inner’, ‘cross’},
df = df_cas.merge(df_fal, on = 'id_persona', how = 'outer')
df['fallecido'].fillna(0, inplace=True)  # Llenar con 0 las personas que no fallecieron en el dataset de casos positivos
df['fallecido'] = df['fallecido'].astype('int8')  # Cambiar 'data type' para reducir tamaño

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fecha_fallecimiento,fallecido,fecha,year,epi_week
0,34978840,20200306.0,2.0,1.0,25.0,1.0,15.0,NaN,0,NaT,NaN,NaN
1,34978840,20220112.0,2.0,1.0,42.0,3.0,15.0,NaN,0,NaT,NaN,NaN
2,12149681,20200307.0,2.0,1.0,25.0,1.0,15.0,NaN,0,NaT,NaN,NaN
3,12149681,20220111.0,2.0,1.0,73.0,6.0,15.0,NaN,0,NaT,NaN,NaN
4,12211040,20200307.0,2.0,1.0,7.0,0.0,15.0,NaN,0,NaT,NaN,NaN


In [ ]:
# Total de casos de casos positivos + información de fallecimiento
len(df)

3570841

In [ ]:
# Número de fallecidos en el dataset de casos postivos
len(df[df.fallecido == 1])

214890

**IMPORTANTE**: Notar que algunos casos aparecen más de una vez (reinfecciones de COVID-19)

In [ ]:
# Eliminar datasets que ya no serán usados
del df_fal, df_cas

In [ ]:
def days_between(d1, d2):
    '''Función que retorna el número de días entre 2 días dados'''
    return abs((d2 - d1).days)

fecha_min = df['fecha'].min()  # Primera fecha reportada en el dataset
fecha_max = df['fecha'].max()  # Última fecha reportada en el dataset

# Obtener el número de días transcurridos a partir de la primera fecha reportada (fecha_min)
df['time_days'] = df['fecha'].apply(lambda row : days_between(row, fecha_min)+1)
df['time_days'] = df['time_days'].fillna(df.time_days.max()).astype('int')

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fecha_fallecimiento,fallecido,fecha,year,epi_week,time_days
0,34978840,20200306.0,2.0,1.0,25.0,1.0,15.0,NaN,0,NaT,NaN,NaN,738
1,34978840,20220112.0,2.0,1.0,42.0,3.0,15.0,NaN,0,NaT,NaN,NaN,738
2,12149681,20200307.0,2.0,1.0,25.0,1.0,15.0,NaN,0,NaT,NaN,NaN,738
3,12149681,20220111.0,2.0,1.0,73.0,6.0,15.0,NaN,0,NaT,NaN,NaN,738
4,12211040,20200307.0,2.0,1.0,7.0,0.0,15.0,NaN,0,NaT,NaN,NaN,738


In [ ]:
# Ordenar columnas
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento',
         'fallecido', 'time_days', 'fecha_fallecimiento', 'year', 'epi_week']]

# Obtener el año y la semana epidemiológica de cada día obtenido
date_life = Week.fromdate(fecha_max).weektuple()
df['year'] = df['year'].fillna(date_life[0]).astype('int')
df['epi_week'] = df['epi_week'].fillna(date_life[1]).astype('int')

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,year,epi_week
0,34978840,20200306.0,2.0,1.0,25.0,1.0,15.0,0,738,NaN,2022,10
1,34978840,20220112.0,2.0,1.0,42.0,3.0,15.0,0,738,NaN,2022,10
2,12149681,20200307.0,2.0,1.0,25.0,1.0,15.0,0,738,NaN,2022,10
3,12149681,20220111.0,2.0,1.0,73.0,6.0,15.0,0,738,NaN,2022,10
4,12211040,20200307.0,2.0,1.0,7.0,0.0,15.0,0,738,NaN,2022,10


### Convertir nuestro dataframe (df) en dask dataframe (ddf) para poder trabajar con el ddf de vacunados

**NOTA**: El dataset de vacunados es demasiado grande como para usarlo como pandas dataframe

In [ ]:
df = dd.from_pandas(df, npartitions=3)

### Dataset de vacunados

In [ ]:
# Leer el dataset (dask dataframe ddf)
url_vac = '../RawData/TB_VACUNACION_COVID19.csv'
col_vac = ['id_persona','fecha_vacunacion','dosis', 'id_vacuna']
ddf_vac = dd.read_csv(url_vac, sep = ",", usecols = col_vac, dtype = {'fecha_vacunacion':'int32', 
                                                                      'dosis': 'int8',
                                                                      'id_vacuna': 'int8',
                                                                      'id_persona': 'int32'})

In [ ]:
ddf_vac.head()

,id_persona,fecha_vacunacion,id_vacuna,dosis
0,13474929,20211127,5,2
1,15456861,20220222,5,3
2,15428123,20211013,1,1
3,17787963,20211014,6,2
4,17873180,20220227,5,1


In [ ]:
"""
# Convertir la columna 'fecha_vacunacion' a formato fecha y obtener el año y semana epidemiológica
ddf_vac = ddf_vac.assign(fecha_vacunacion = dd.to_datetime(ddf_vac['fecha_vacunacion'], 
                                                           format = '%Y%m%d', 
                                                           errors="coerce"))
"""
# Renombrar y ordenar columnas
ddf_vac = ddf_vac.rename(columns = {'fecha_vacunacion': 'fecha_vacuna'})
ddf_vac = ddf_vac[['id_persona', 'dosis', 'fecha_vacuna', 'id_vacuna']]

In [ ]:
ddf_vac.head()

,id_persona,dosis,fecha_vacuna,id_vacuna
0,13474929,2,20211127,5
1,15456861,3,20220222,5
2,15428123,1,20211013,1
3,17787963,2,20211014,6
4,17873180,1,20220227,5


In [ ]:
# TOTAL de vacunas aplicadas
len(ddf_vac)

63386865

### Separar información de dosis recibidas para juntarlos con nuestro dataframe principal (df)

In [ ]:
# Personas que recibieron 1era dosis
df_dosis_1 = ddf_vac[ddf_vac.dosis == 1]
df_dosis_1 = df_dosis_1.rename(columns = {'dosis' : 'dosis_1', 
                                          'fecha_vacuna': 'fecha_vacuna_1', 
                                          'id_vacuna': 'id_vacuna_1'})
len(df_dosis_1)

27948872

In [ ]:
# Unir a nuestro df principal
df = df.merge(df_dosis_1, on = 'id_persona', how = 'left')
del df_dosis_1

In [ ]:
# Personas que recibieron 2nda dosis
df_dosis_2 = ddf_vac[ddf_vac.dosis == 2]
df_dosis_2['dosis'] = df_dosis_2['dosis'].replace(2, 1)
df_dosis_2 = df_dosis_2.rename(columns = {'dosis' : 'dosis_2', 
                                          'fecha_vacuna': 'fecha_vacuna_2', 
                                          'id_vacuna': 'id_vacuna_2'})
len(df_dosis_2)

24789831

In [ ]:
# Unir a nuestro df principal
df = df.merge(df_dosis_2, on = 'id_persona', how = 'left')
del df_dosis_2

In [ ]:
# Personas que recibieron 3era dosis
df_dosis_3 = ddf_vac[ddf_vac.dosis == 3]
df_dosis_3['dosis'] = df_dosis_3['dosis'].replace(3, 1)
df_dosis_3 = df_dosis_3.rename(columns = {'dosis' : 'dosis_3', 
                                          'fecha_vacuna': 'fecha_vacuna_3', 
                                          'id_vacuna': 'id_vacuna_3'})
len(df_dosis_3)

10647688

In [ ]:
# Unir a nuestro df principal
df = df.merge(df_dosis_3, on = 'id_persona', how = 'left')   
del df_dosis_3, ddf_vac

In [ ]:
# TOTAL de casos positivos más vacunación
len(df)

3570867

### Convertir nuestro ddf en un pandas dataframe

In [ ]:
# IMPORTANTE: Verificar cantidad de RAM disponible antes de ejecutar esta celda
df = df.compute()

In [ ]:
df.head()

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,epi_week,dosis_1,fecha_vacuna_1,id_vacuna_1,dosis_2,fecha_vacuna_2,id_vacuna_2,dosis_3,fecha_vacuna_3,id_vacuna_3
0,10629488,20200307.0,2.0,1.0,51.0,4.0,15.0,0,738,NaN,...,10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,20211129.0,5.0
1,10629488,20220112.0,2.0,2.0,27.0,1.0,14.0,0,738,NaN,...,10,NaN,NaN,NaN,NaN,NaN,NaN,1.0,20211129.0,5.0
2,5786491,20200315.0,2.0,2.0,39.0,2.0,15.0,0,738,NaN,...,10,1.0,20210823.0,6.0,1.0,20210918.0,6.0,1.0,20220112.0,5.0
3,5786491,20220111.0,2.0,2.0,25.0,1.0,4.0,0,738,NaN,...,10,1.0,20210823.0,6.0,1.0,20210918.0,6.0,1.0,20220112.0,5.0
4,9040259,20200317.0,2.0,1.0,28.0,1.0,15.0,0,738,NaN,...,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Personas con 1 dosis recibida en el dataset de casos positivos
len(df[df.dosis_1 == 1])

3153441

In [ ]:
# Personas con 2 dosis recibidas en el dataset de casos positivos
len(df[df.dosis_2 == 1])

3020939

In [ ]:
# Personas con 3 dosis recibidas en el dataset de casos positivos
len(df[df.dosis_3 == 1])

1752180

In [ ]:
# Verificamos que se encuentren todas las columnas
df.columns

Index(['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat',
       'departamento', 'fallecido', 'time_days', 'fecha_fallecimiento', 'year',
       'epi_week', 'dosis_1', 'fecha_vacuna_1', 'id_vacuna_1', 'dosis_2',
       'fecha_vacuna_2', 'id_vacuna_2', 'dosis_3', 'fecha_vacuna_3',
       'id_vacuna_3'],
      dtype='object')

### Rellenar espacios vacíos con 0

In [ ]:
# Crear una columna llamado 'vacunado' si recibe 2 o 3 dosis
df['vacunado'] = np.where((df['dosis_2'] == 1) | (df['dosis_3'] == 1), 1, 0).astype('int8')

In [ ]:
# Convertir de float a int para ocupar menos espacio en memoria
df['dosis_1'] = df['dosis_1'].fillna(0).astype('int8')
df['dosis_2'] = df['dosis_2'].fillna(0).astype('int8')
df['dosis_3'] = df['dosis_3'].fillna(0).astype('int8')

df['id_vacuna_1'] = df['id_vacuna_1'].fillna(0).astype('int8')
df['id_vacuna_2'] = df['id_vacuna_2'].fillna(0).astype('int8')
df['id_vacuna_3'] = df['id_vacuna_3'].fillna(0).astype('int8')

In [ ]:
# Verificamos que la columna vacunados tenga en cuenta 2 o 3 dosis como persona vacunada
df[['dosis_1', 'dosis_2', 'dosis_3', 'vacunado']].head(10)

,dosis_1,dosis_2,dosis_3,vacunado
0,0,0,1,1
1,0,0,1,1
2,1,1,1,1
3,1,1,1,1
4,0,0,0,0
5,1,1,1,1
6,1,1,1,1
7,1,1,1,1
8,0,0,1,1
9,1,1,0,1


### Ordenar y Guardar el df

In [ ]:
df = df[['id_persona', 'fecha_resultado', 'metododx', 'sexo', 'edad', 'edad_cat', 'departamento', 
         'fallecido', 'time_days', 'fecha_fallecimiento', 'year', 'epi_week',
         'vacunado', 'dosis_1', 'dosis_2', 'dosis_3', 
         'id_vacuna_1', 'id_vacuna_2', 'id_vacuna_3' ,'fecha_vacuna_1','fecha_vacuna_2', 'fecha_vacuna_3']]

### Eliminar duplicados

In [ ]:
# Ordenar de acuerdo al id_de persona y fecha de resultado
df = df.sort_values(by = ['id_persona', 'fecha_resultado'])

In [ ]:
# Vemos todos los duplicados, ya que debería haber un solo id para cada persona
df[df.id_persona.duplicated() == True]

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
7809,47,20220129.0,2.0,1.0,62.0,5.0,25.0,0,738,NaN,...,1,1,1,1,1,2,5,20210506.0,20210608.0,20211127.0
61718,89,20220129.0,2.0,2.0,74.0,6.0,25.0,0,738,NaN,...,1,1,1,0,1,2,0,20210426.0,20210609.0,NaN
39309,227,20210421.0,3.0,1.0,81.0,7.0,25.0,0,738,NaN,...,1,1,1,0,6,6,0,20211202.0,20211223.0,NaN
43320,324,20210525.0,3.0,2.0,54.0,4.0,25.0,0,738,NaN,...,1,1,1,1,2,1,5,20210824.0,20210927.0,20220106.0
17604,555,20220127.0,2.0,2.0,55.0,4.0,25.0,0,738,NaN,...,1,1,1,1,2,2,5,20210712.0,20210810.0,20211228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25527,37278364,20220116.0,3.0,2.0,1.0,0.0,20.0,0,738,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
33172,37281588,20210321.0,2.0,1.0,19.0,1.0,15.0,0,738,NaN,...,0,1,0,0,5,0,0,20220208.0,NaN,NaN
80,37285950,20220112.0,2.0,1.0,26.0,1.0,20.0,1,65,20200506.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
1927,37286078,20220112.0,2.0,2.0,22.0,1.0,15.0,1,90,20200531.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


Podremos conservar el último duplicado, ya es es la fecha más reciente de un caso positivo. Estos duplicados representan varios contagios de la misma persona, sin embargo se toman como un nuevo contagio.

En este caso para el propósito del estudio nos interesa la mortalidad y la vacunación, por ello no se toma en cuenta el número de veces de contagio o fechas de la misma, si no todos los datos e id del paciente.

In [ ]:
# Eliminamos los duplicados
df = df.drop_duplicates('id_persona', keep='last') \
       .sort_values(by = ['fecha_resultado']) \
       .reset_index(drop = True)

In [ ]:
df.head(10)

,id_persona,fecha_resultado,metododx,sexo,edad,edad_cat,departamento,fallecido,time_days,fecha_fallecimiento,...,vacunado,dosis_1,dosis_2,dosis_3,id_vacuna_1,id_vacuna_2,id_vacuna_3,fecha_vacuna_1,fecha_vacuna_2,fecha_vacuna_3
0,13708801,20200309.0,2.0,2.0,15.0,0.0,10.0,0,738,NaN,...,1,1,1,1,5,5,5,20210601.0,20210622.0,20211203.0
1,2020432,20200310.0,2.0,1.0,46.0,3.0,15.0,0,738,NaN,...,1,1,1,1,5,5,5,20210517.0,20210607.0,20211206.0
2,5493803,20200311.0,2.0,2.0,28.0,1.0,15.0,0,738,NaN,...,1,0,0,1,0,0,5,NaN,NaN,20220104.0
3,7979960,20200311.0,2.0,2.0,21.0,1.0,15.0,0,738,NaN,...,1,1,1,0,6,6,0,20210926.0,20211019.0,NaN
4,27127031,20200312.0,2.0,1.0,24.0,1.0,15.0,0,738,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
5,35668643,20200312.0,2.0,2.0,39.0,2.0,15.0,0,738,NaN,...,1,0,0,1,0,0,5,NaN,NaN,20211201.0
6,10960790,20200312.0,2.0,2.0,58.0,4.0,15.0,0,738,NaN,...,1,1,1,1,5,5,5,20210625.0,20210716.0,20211222.0
7,4102088,20200312.0,2.0,2.0,37.0,2.0,15.0,0,738,NaN,...,1,1,1,1,6,6,5,20210918.0,20211009.0,20220127.0
8,36889651,20200312.0,2.0,2.0,75.0,6.0,15.0,1,18,20200320.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN
9,36611013,20200312.0,2.0,1.0,78.0,6.0,15.0,1,17,20200319.0,...,0,0,0,0,0,0,0,NaN,NaN,NaN


In [ ]:
# Verificamos columnas y data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353131 entries, 0 to 3353130
Data columns (total 22 columns):
 #   Column               Dtype  
---  ------               -----  
 0   id_persona           int32  
 1   fecha_resultado      float64
 2   metododx             float64
 3   sexo                 float64
 4   edad                 float64
 5   edad_cat             float64
 6   departamento         float64
 7   fallecido            int8   
 8   time_days            int64  
 9   fecha_fallecimiento  float64
 10  year                 int64  
 11  epi_week             int64  
 12  vacunado             int8   
 13  dosis_1              int8   
 14  dosis_2              int8   
 15  dosis_3              int8   
 16  id_vacuna_1          int8   
 17  id_vacuna_2          int8   
 18  id_vacuna_3          int8   
 19  fecha_vacuna_1       float64
 20  fecha_vacuna_2       float64
 21  fecha_vacuna_3       float64
dtypes: float64(10), int32(1), int64(3), int8(8)
memory usage: 370.9 MB

In [ ]:
# Guardamos en un csv
df.to_csv('../Data/DP3_covid19-peru_casos-positivos_+_info-fallecimiento_+_info-vacunas.csv',  index=False)

In [ ]:
# Eliminamos el dataframe en memoria
del df